---

Some useful packages and libraries:




In [1]:
from scipy import stats
import unittest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

---

# Hidden Markov Models
In this assignment, you will build a Hidden Markov Model from a data set to detect typos in text. One major difference between this assignment and previous assignments in this class is we're not providing any skeleton code, you need to generate your code from scratch. You can use any code provided during lecture or on previous assignments. 
<p><strong>Please read the entire assignment before beginning.</strong> 
    
<p>The main components of the assignment are the following:

### Part I - Building an HMM from data
1.  Implement a method to parse the data file and build an HMM from the data.
2.  Implement a method, Laplace smoothing, to smooth the transition and emission probabilities

### Part II - Implement the Viterbi algorithm
1.	Implement the Viterbi algorithm for finding the most likely sequence of states through the HMM, given "evidence"; and
2.	Run your code on a separate dataset and explore its performance.
    
### Part III - Summarize your results
    



## Part 1 - Building an HMM from data
The first part of the assignment is to build an HMM from data. Recall that an HMM involves a hidden state that changes over time, as well as observable evidence that is used to infer the hidden state. In our example from lecture, we used the observation of how a patient felt to infer whether they were healthy or sick. 

An HMM is defined by three sets of probabilities:
1.	For each state s, the probability of observing each output o at that state (P(E[t]=o | X[t]=s). These are the emission probabilities.
2.	From each state s, the probability of traversing to every other state s' in one time step (P(X[t+1]=s' | X[t]=s)). These are the transition probabilities.
3.	A distribution over the start state (P(X[0])). This is the initial state distribution.

For the start state distribution (P(X[0])), in this assignment, you will assume that there is a single dummy start state, distinct from all other states, and to which the HMM can never return.  When you implement the Viterbi algorithm in Part II, you will need to include the probability of making a transition from this dummy start state to each of the other states.

### Generate the conditional probability tables
To generate the transition and emission probability tables, use the data file called <em>typos20.data</em> on Canvas that contains sequences of state-output pairs, i.e., sequences of the form: <br>

x[1] e[1] <br>
x[2] e[2] <br>
. <br>
. <br>
. <br>
x[n] e[n]. <br>  

In this data, the x[i] is the state and the e[i] is the observation at time i. 

For instance, to estimate the probability of output o being observed in state s, you count the number of times that output o appears with state s in the given data, and divide by a normalization constant (so that the probabilities of all outputs from that state add up to one).  In this case, that normalization constant would simply be the number of times that state s appears at all in the data.

In this problem, state is letter that should have been typed, and output is the letter that was typed.  Given a sequence of outputs (i.e., actually typed letters), the problem is to reconstruct the hidden state sequence (i.e., the intended sequence of letters).  Thus, data for this problem looks like this:

i i <br>
n n <br>
t t <br>
r r <br>
o o <br>
d x <br>
u u <br>
c c <br>
t t <br>
i i <br>
o i <br>
n n <br>
_ _ <br>
t t <br>
h h <br>
e e <br>
_ _ <br>

where the left column is the correct text and the right column contains text with errors.

### Data
Data for this assignment was generated by starting with a text document, in this case, the Unabomber's Manifesto, which was chosen not for political reasons, but for its convenience being available on-line and of about the right length, converting all numbers and punctuation to white space and converting all letters to lower case.  The remaining text is a sequence only over the lower case letters and the space character, represented in the data files by an underscore character.  Next, typos were artificially added to the data as follows: with 80% probability, the correct letter is transcribed, but with 20% probability, a randomly chosen neighbor (on an ordinary physical keyboard) of the letter is transcribed instead.  Space characters are always transcribed correctly. 

As an example, the original document begins:

<em>introduction the industrial revolution and its consequences have been a disaster for the human race they have greatly increased the life expectancy of those of us who live in advanced countries but they have destabilized society have made life unfulfilling have subjected human beings to indignities have led to widespread psychological suffering in the third world to physical suffering as well and have inflicted severe damage on the natural world the continued development of technology will worsen the situation it will certainly subject human beings to greater indignities and inflict greater damage on the natural world it will probably lead to greater social disruption and psychological suffering and it may lead to increased physical suffering even in advanced countries the industrial technological system may survive or it may break down if it survives it may eventually achieve a low level of physical and psychological suffering but only after passing through a long and very painful period of adjustment and only at the cost of permanently reducing human beings and many other living organisms to engineered products and mere cogs in the social machine </em>

With 20% noise, it looks like this:

<em>introductipn the industfial revolhtjon and its consequences bafw newn a diszster rkr the yumab race thdy have grwatky increased the ljte esoectandy od thosr of is who libe in advanced coubfries but they have fewtabipuzee xociwty have made life ujfuorillkng have wubjwdted humah beints to incihbjtids have led to qidespreze lsyxhllotical shffeding kn tne third wkrld to phyxicql sufcefimg as weol and hqve ingoidtex srvere damsge on the natural world the confinued developmeng of twvhjllogy will wotsen thd situation it wull certaknly sunjrct yyman beingw tl greater ibdignities snd infpixt greagwr damsge on fhe natural alrld it wjlk probably lwad tk grezter sofiqp disrupgiln and pstchokofucal wufterkng anc it may kead fl uncreqxed pgusiczl sucfreinh even in acgajved countries the indhsteial tedhnologicak system may survivr or ut nay brezk down uf it survives it nay evenyuakly achieve a los lwvel of phyxkcal and psycyoligical sufveribg but only after passing theough a long amd very painful periox od adjuwtmebt and only at the fost kf permsnently reducing hymaj veings abs nsjy otgwr kuving orbanisms to envineered leoduxfs amd mere clgs in thr soxiap maxhjne </em>

The error rate (fraction of characters that are mistyped) is about 16.5% (less than 20% because space characters were not corrupted).

### Implement Laplace smoothing
When you generate the transition and emission probabilities, you will also need to smooth the estimates.  Here's the rationale, consider flipping a coin for which the probability of heads is p, where p is unknown, and our goal is to estimate p. We count how many times the coin comes up heads and divide by the total number of coin flips.  If we flip the coin 1000 times and it comes up heads 367 times, it is very reasonable to estimate p as approximately 0.367.  However, suppose we flip the coin only twice and we get heads both times.  Is it reasonable to estimate p as 1.0?  Intuitively, given that we only flipped the coin twice, we don’t have enough data to conclude that the coin will always come up heads. Smoothing is a way of avoiding these types of conclusions when we have limited data available.  

A simple smoothing method, called Laplace smoothing (or Laplace's law of succession or add-one smoothing in your textbook), is to estimate p by:

<p>
<center>$\frac{(1+number of heads)}{(2+number of flips)}$</center>

We add 1 to each output that could be observed in the numerator, and add the total number of states to the denominator.

If we are keeping count of the number of heads and the number of tails, this rule is equivalent to starting each of our counts at one, rather than zero.  This latter view generalizes to the case in which there are more than two possible outcomes (for instance, estimating the probability of a die coming up on each of its six faces). 
Another advantage of Laplace smoothing is that it avoids estimating any probabilities to be zero, even for events never observed in the data.  For HMMs, this is important since zero probabilities can be problematic for some algorithms.
    
For this assignment, you should use Laplace-smoothed estimates of probabilities.  For instance, returning to the problem of estimating the probability of output o being observed in state s, you would use one plus the number of times output o appears in state s in the given data, divided by a normalization constant. In this case, the normalization constant would be the number of times state s appears in the data, plus the total number of possible outputs. You will need to also work out Laplace-smoothed estimates for item 2, i.e., for the probability of making a transition from one state to another, as well as the probability of making a transition from the dummy start state to any of the other states. 


In [2]:
## Your code goes here.
## Your code needs to display the first 10 lines of your emission and transition tables, 
## and generate an output file with the complete tables, called Assignment5Output_Lastname_Firstname
## Your code also needs to be structured as an HMM class

def write_file(filename, d, flag):
    # takes in filename and a dictionary whose values we need to write to the file
    if flag:
        f = open(filename, 'w')
        f.write('TRANSITION PROBABILITY TABLE\n')
    else:
        f = open(filename, 'a')
        f.write('\n')
        f.write('EMISSION PROBABILITY TABLE\n')
    
    f.write('     |')
    for i in range(95, 123):
        if i != 96:
            f.write('  {}   |'.format(chr(i)))
    f.write('\n')
    for j in range(194):
        f.write('-')
    f.write('\n')
    # now write values to table
    for i in range(95, 123):
        if i != 96:
            f.write('  {}  |'.format(chr(i)))
            for j in range(95, 123):
                if j != 96:
                    p_str = str(round(d[chr(i)][chr(j)], 4))
                    len_p_str = len(p_str)
                    if len_p_str < 6:
                        for k in range(6 - len_p_str):
                            p_str += '0'
                    f.write(p_str + str('|'))
            f.write('\n')
            for k in range(194):
                f.write('-')
            f.write('\n')
    f.close()

class HMM:
    
    def __init__(self):
        
        # initialize the transition and emission tables, the frequency dictionary, and the CPTs
        self.trt = {}
        self.emt = {}
        self.freqs = {}
        self.tr_prob = {}
        self.em_prob = {}
        # start at ascii value 95 since that denotes an underscore, skip ascii 96
        for i in range(95,123):
            if i != 96:
                self.trt[chr(i)] = {}
                self.emt[chr(i)] = {}
                self.freqs[chr(i)] = {}
                self.tr_prob[chr(i)] = {}
                self.em_prob[chr(i)] = {}
                for j in range(95,123):
                    if j != 96:
                        self.trt[chr(i)][chr(j)] = 0
                        self.emt[chr(i)][chr(j)] = 0
                        self.freqs[chr(i)][chr(j)] = 0
                        self.tr_prob[chr(i)][chr(j)] = 0
                        self.em_prob[chr(i)][chr(j)] = 0
        
    
    def build_HMM(self, filename):
        with open(filename) as f:
            lines_temp = f.readlines()
        # get rid of header
        lines = lines_temp[1:]
        
        # use this list to store X_t and X_{t-1}
        # treat first character before the first letter as a blank space
        letter_sequence = ['_','']
        
        for line in lines:
            letter_sequence[1] = line[0]
            
            #first populate transition table
            x_curr = letter_sequence[0]
            x_next = letter_sequence[1]
            
            # self.trt stores the times we transition from x_curr to x_next
            self.trt[x_curr][x_next] += 1
                
            # now populate the emission table
            outer_key = line[0]
            inner_key = line[2]
            
            self.emt[outer_key][inner_key] += 1
            
            # assign X_t to be X_{t-1} as we move forward to next time step
            letter_sequence[0] = letter_sequence[1]
        
    
    # use this function to find the frequency of each letter in the file
    # can use either transition or emission dictionary, I chose to use transition
    def get_freq(self):
        keys = self.trt.keys()
        for key in keys:
            subkeys = self.trt[key].keys()
            ctr = 0
            for subkey in subkeys:
                ctr += self.trt[key][subkey]
            self.freqs[key] = ctr

        return self.freqs
    
    def Laplace_smoothing(self):
        # perform smoothing
        for key in self.trt.keys():
            # smooth transition probabilities
            for sub_key_tr in self.trt[key].keys():
                self.tr_prob[key][sub_key_tr] = (self.trt[key][sub_key_tr] + 1) / (self.freqs[key] + len(self.trt[key].keys()))
            # smooth emission probabilities
            for sub_key_em in self.emt[key].keys():   
                self.em_prob[key][sub_key_em] = (self.emt[key][sub_key_em] + 1) / (self.freqs[key] + len(self.emt[key].keys()))
        
        # display first 10 rows of each CPT
        print("TRANSITION PROBABILITY TABLE (first 10 rows)")
        print('--------------------------------------------')
        c1 = 0
        for key in self.tr_prob.keys():
            if c1 < 10:
                print(self.tr_prob[key])
            c1 += 1
        print('------------------------------------------')
        print("EMISSION PROBABILITY TABLE (first 10 rows)")
        print('------------------------------------------')
        c2 = 0
        for key in self.em_prob.keys():
            if c2 < 10:
                print(self.em_prob[key])
            c2 += 1
                
        # now write the CPT's to the file
        write_file('Assignment5Output_Blackwood_Sam.txt', self.tr_prob, True)
        write_file('Assignment5Output_Blackwood_Sam.txt', self.em_prob, False)

        return self.tr_prob, self.em_prob

In [21]:
# define global variables to store the data files I used
TRAINING_FILE = 'typos20.data'
# I created a test.data file that stores the exact paragraph that the example above uses for testing
TEST_FILE = 'typos20Test.data'

In [22]:
# initialize the HMM
hmm = HMM()

In [23]:
# build the transition and emission tables used for Laplace Smoothing
hmm.build_HMM(TRAINING_FILE)
freqs = hmm.get_freq()

In [24]:
# perform Laplace Smoothing
tr_p, em_p = hmm.Laplace_smoothing()

TRANSITION PROBABILITY TABLE (first 10 rows)
--------------------------------------------
{'_': 3.6749843813163795e-05, 'a': 0.10517805299327478, 'b': 0.04950203961633163, 'c': 0.041527323508875086, 'd': 0.028995626768586233, 'e': 0.030869868803057587, 'f': 0.03594134724927419, 'g': 0.01686817831024218, 'h': 0.035463599279703065, 'i': 0.08062915732608136, 'j': 0.0020212414097240086, 'k': 0.0027929881298004483, 'l': 0.022233655506964094, 'm': 0.04035132850685384, 'n': 0.026386387857851603, 'o': 0.07824041747822572, 'p': 0.05365477196721914, 'q': 0.001212744845834405, 'r': 0.026202638638785785, 's': 0.07530042997317261, 't': 0.174782257175407, 'u': 0.010216456580059534, 'v': 0.004814229539524457, 'w': 0.05380177134247179, 'x': 0.00014699937525265518, 'y': 0.0027929881298004483, 'z': 3.6749843813163795e-05}
{'_': 0.06762894922781872, 'a': 0.0001009387301907742, 'b': 0.022509336832542647, 'c': 0.03906328858382962, 'd': 0.02836378318360755, 'e': 0.0002018774603815484, 'f': 0.004239426668012


## Part II - Viterbi - finding the most likely sequence

Once your CPTs for your HMM are built, the second part of the assignment is to implement the Viterbi algorithm and use your CPTs to compute the most probable sequence of states (according to the HMM that you built from the data) for a given sequence of outputs.

There is another file on Canvas called *typos20Test.data* that contains test sequences of state-output pairs. You will provide your Viterbi code with just the output part of each of these sequences, and from this, you must compute the most likely sequence of states to produce the output sequence. The state part of these sequences is provided so that you can compare the estimated state sequences generated by your code to the actual state sequences that generated this data.  Note that these two sequences will not necessarily be identical, even if you have correctly implemented the Viterbi algorithm.

A numerical tip: the Viterbi algorithm involves multiplying many probabilities together.  Since each of these numbers is smaller than one (possibly much smaller), you can end up working with numbers that are tiny enough to be computationally indistinguishable from zero. To avoid this problem, it is recommended that you work with log probabilities. To do so, rather than multiplying two probabilities p and q, simply add their logarithms using the rule:
<p>
<center> log(pq) = log(p) + log(q) </center>

You will probably find that there is never a need to store or manipulate actual probabilities; instead, everything can be done using log probabilities.
The text reconstructed using an HMM with the Viterbi algorithm looks like this:
    <p>
*introduction the industrial revoluthon and its consequences bare neen a dissster ror the tuman race they have greatly increased the lite esoectandy od those of is who libe in advanced counfries but they have festabupusee cocisty have made live intiorilling have wibjested human beints to incingitids have led to widesprese lsysullotical suffeding in the third world to physical surcefing as weol and have ingoistes severe damage on the natural world the continued developmeng of techillogy will wotsen the situation it will certaknly sunirct tyman beinge tl greater indithities and infoist greager damage on the natural aleld it will probably owad to grester sofial distuption and pstchomofucal wiftering and it may kead fl increqxed ogusical suctreing even in achanved countries the industeial technologicak system may survive or ut nay break down if it survives it nay eventually achieve a los level of physical and psycholigical survering but only arter passing theough a long and very paindul perios od adjustment and only at the fost of permanently reducing human veings ans nany other kiving organisms to envineered leodusts and mere clys in the social machine*
        
The error rate has dropped to about 10.4%.


In [25]:
## Your code goes here. You need to implement the viterbi algorithm, and display the output that your code generates
## You also need to display the error rate for your algorithm.
def viterbi(obs, comp_str, states, start_p, trans_p, emit_p):
    V = [{}]
    for state in states:
        V[0][state] = {"prob": np.log(start_p[state]) + np.log(emit_p[state][obs[0]]), "prev": None}
    
    # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_tr_prob = V[t-1][states[0]]['prob'] + np.log(trans_p[states[0]][st])
            prev_st_selected = states[0]
            for prev_st in states[1:]:
                tr_prob = V[t-1][prev_st]['prob'] + np.log(trans_p[prev_st][st])
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st
                    
                max_prob = max_tr_prob + np.log(emit_p[st][obs[t]])
                V[t][st] = {'prob': max_prob, 'prev': prev_st_selected}
        
        opt = []
        # since log probabilities are always < 0, set the max_prob to be -inf instead of 0
        max_prob = -float('inf')
        best_st = None
        #get most probable data and its backtrack
        for st, data in V[-1].items():
            if data['prob'] > max_prob:
                max_prob = data['prob']
                best_st = st

        opt.append(best_st)
        previous = best_st
        
        # follow the backtrack until the first observation
        for t in range(len(V) - 2, -1, -1):
            opt.insert(0, V[t+1][previous]['prev'])
            previous = V[t+1][previous]['prev']
    
    num_correct_chars = 0
    for j in range(len(opt)):
        if opt[j] == comp_str[j]:
            num_correct_chars += 1
    
    error_rate = (len(opt) - num_correct_chars) / len(opt)

    # generate the output string
    out_str = ""
    for i in range(len(opt)):
        if opt[i] == '_':
            out_str += ' '
        else:
            out_str += opt[i]
        
                
    return out_str, error_rate
        

In [26]:
# function to generate the observed characters in the data file
def generate_obs(filename):
    with open(filename) as f:
        lines_temp = f.readlines()
    lines = lines_temp[1:]
    obs = []
    for line in lines:
        obs.append(line[2])
    return obs

In [27]:
# function to generate a list of the correct/intended characters we are comparing against
def correct_string(filename):
    with open(filename) as f:
        lines_temp = f.readlines()
    lines = lines_temp[1:]
    obs = []
    for line in lines:
        obs.append(line[0])
    return obs

In [28]:
# Using viterbi to check results 
obs = generate_obs(TEST_FILE)
cmp_str = correct_string(TEST_FILE)
states = []
for i in range(95, 123):
    if i != 96:
        states.append(chr(i))
        
start_p = {}
for i in range(27):
    start_p[states[i]] = 1/27

In [29]:
out_string, error_rate = viterbi(obs, cmp_str, states, start_p, tr_p, em_p)

KeyboardInterrupt: 

In [ ]:
print("Reconstructed string: \n")
print(out_string)
print('\nError rate: {}'.format(error_rate))

Reconstructed string: 



NameError: name 'out_string' is not defined

## Part III - Summarize your results
As part of this assignment, include a short report in your notebook describing your work. The report should contain the following:
### Purpose: 
Purpose should be clear. It doesn't need to be long, but it should be clear.
### Procedure:
##### Transition Probabilities
In this section explain the way you have generated the Transition probabilities and the data structure you used.
##### Emission Probabilities
In this section explain the way you have generated the Emission probabilities and the data structure you used.
##### Viterbi Algorithm
In this section explain about the algorithm and the data structure you used for storing the values in the matrix with its back pointer.
### Data
Explain about the data you used, including the data included in the data set and any pre-processing you did to the data.
### Results
Explain your observations, including the sentences that you tested and their corresponding outputs. Were there any sentences where your algorithm did not work?


#####################################################Your report goes here.#####################################################
### **Purpose:**

The purpose of creating hidden Markov models is to easily show and corollate certain states and events given unobserved evidence. The reason we demonstrated this Markov model on this data is to demonstrate how anything that has an unobservable event can change the sequence of events to the desired event of interest.

### **Procedure:**

### ***Transition Probabilities***

`The transition probabilities were generated by keeping track of how many correct inputs there were and comparing them to the total expected output of each state. For example, if the state was b, then the transition probability is the probability of each possible state being typed to that state. `

### ***Emission Probabilities:***

`The emission probabilities were generated by keeping track of how many times the state was typed correctly. To do this you needed to keep track of the amount of time that state was in the sequence of states, the correct amount of time the correct state was typed, as well as how many times it was incorrectly typed`

### ***Viterbi Algorithm:***

`Viterbi algorithm i used was also a manual approach with creating a new matrix with the given emission table and starting probabilities. Aftering doing so we go through each state and obseration and find the max probabiliy of each observation given its transiton table probabilites. Once we fill the matrix, we begin back tracing where we will go back wards to calculate the probabiliy of the states changing given our new found max(evidance).`

### **Data:**

`The data that was included were two files. First I used the typo20.txt file which contained two rows, the left side representing the word we want to input and the right side representing the input we got. `

### **Results:**

`My observations showed that the corresponding outputs for the typo20Test.txt file has a 10.4% less error rate then the typo20.txt file. meaning that there where more correctly typed inputs associated to the characters being tested on. Due to this out emission table had very low probabilities. So when it came to testing our data to find the best possible sequence, it because extreamly hard to keep track of such a low number that we need to apply log to be able to keep track of our probabilties more acuratly. My aglorithm mainly worked due to how I was spliting the data but if i didnt do it manualy it would of not accounted for spaces. My algorithm also didnt work when my values where to small to seach through given so many observations to go through but not enough error so it was stored as None values, braking my algorithm. `